## Установка Tensorflow 

``` pip install tensorflow==2.0.0-beta1  ```

In [3]:
import tensorflow as tf
tf.__version__

'2.0.0-beta1'

In [56]:
a = tf.constant([1,2,3], dtype=tf.float32)
b = tf.zeros([3])+5
c = a + b
c.numpy()

array([6., 7., 8.], dtype=float32)

In [58]:
def np_sum_squares(N):
    return np.sum(np.arange(N)**2)

In [64]:
%%timeit
np_sum_squares(10**8)

1.28 s ± 70.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [65]:
def tf_sum_squares(N):
    N = tf.constant(N, dtype='int64')
    return tf.reduce_sum((tf.range(N)**2))

In [66]:
%%timeit
tf_sum_squares(10**8)

807 ms ± 34.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [68]:
@tf.function
def tf_sum_squares(N):
    N = tf.constant(N, dtype='int64')
    return tf.reduce_sum((tf.range(N)**2))

In [69]:
%%timeit
tf_sum_squares(10**8)

286 µs ± 56.9 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [70]:
tf.keras.__version__

'2.2.4-tf'

In [72]:
celsius    = np.array([-40, -10,  0,  8, 15, 22,  38],  
                      dtype='float32')

fahrenheit = np.array([-40,  14, 32, 46, 59, 72, 100],
                       dtype='float32')

In [75]:
# параметры модели 
a = tf.Variable(tf.random.normal([1]), name='bias')
b = tf.Variable(tf.random.normal([1]), name='k')

print(a.numpy())
print(b.numpy())

[-1.1439809]
[1.496614]


In [77]:
# Наша модель
def linear_regression(x):
    return a + b*x

linear_regression(celsius)

<tf.Tensor: id=81352, shape=(7,), dtype=float32, numpy=
array([-61.00854  , -16.11012  ,  -1.1439809,  10.828931 ,  21.30523  ,
        31.781527 ,  55.72735  ], dtype=float32)>

In [79]:
# Ошибка для модели
def mean_square(y_pred, y_true):
    return tf.reduce_mean((y_pred-y_true)**2)

y_pred = linear_regression(celsius)
mean_square(y_pred, fahrenheit)

<tf.Tensor: id=81369, shape=(), dtype=float32, numpy=1240.285>

In [90]:
# оптимизатор 
optimizer = tf.optimizers.SGD(learning_rate=0.001)

# процесс оптимизации
def model_train(X, Y):

    # Указываем нашему дифириенцированию 
    # за какими переменными следить 
    with tf.GradientTape() as g:
        pred = linear_regression(X)
        loss = mean_square(pred, Y)

    # Вычисляем градиенты
    gradients = g.gradient(loss, [a, b])
    # print(gradients)
    
    # Обновляем веса a и b в ходе одной итерации спуска 
    optimizer.apply_gradients(zip(gradients, [a, b]))
    pass

In [91]:
model_train(celsius, fahrenheit)

In [92]:
# Обучение
epochs = 2000 # число эпох 

for i in range(epochs):
    
    # Делаем щаг градиентного спуска 
    model_train(celsius, fahrenheit)
    
    # Каждую сотую итерацию следим за тем, что произошло
    if i%100 == 0:
        y_pred = linear_regression(celsius)
        loss_val = mean_square(y_pred, fahrenheit)
        print("step: %i, loss: %f, a: %f, b: %f" % (i, loss_val, a.numpy(), b.numpy()))


step: 0, loss: 1027.646240, a: -0.759606, b: 2.074015
step: 100, loss: 699.663757, a: 4.961123, b: 2.025738
step: 200, loss: 476.366180, a: 9.681405, b: 1.985902
step: 300, loss: 324.340576, a: 13.576196, b: 1.953032
step: 400, loss: 220.838242, a: 16.789862, b: 1.925911
step: 500, loss: 150.371597, a: 19.441521, b: 1.903533
step: 600, loss: 102.396172, a: 21.629465, b: 1.885069
step: 700, loss: 69.733795, a: 23.434763, b: 1.869833
step: 800, loss: 47.496433, a: 24.924353, b: 1.857262
step: 900, loss: 32.356762, a: 26.153442, b: 1.846889
step: 1000, loss: 22.049257, a: 27.167597, b: 1.838331
step: 1100, loss: 15.031702, a: 28.004393, b: 1.831268
step: 1200, loss: 10.254022, a: 28.694847, b: 1.825442
step: 1300, loss: 7.001275, a: 29.264553, b: 1.820634
step: 1400, loss: 4.786748, a: 29.734627, b: 1.816667
step: 1500, loss: 3.279042, a: 30.122496, b: 1.813393
step: 1600, loss: 2.252561, a: 30.442533, b: 1.810692
step: 1700, loss: 1.553717, a: 30.706602, b: 1.808464
step: 1800, loss: 1.0

## Собираем свой w2v! 

In [5]:
import gensim
gensim.__version__

'3.7.1'

In [6]:
w2v = gensim.models.KeyedVectors.load_word2vec_format("184/model.bin",
                                                      binary=True)

In [9]:
w2v.most_similar('митинг_NOUN')

[('митинг_PROPN', 0.5869618654251099),
 ('пикет_NOUN', 0.5843158960342407),
 ('шествие_NOUN', 0.5592673420906067),
 ('верхнепышминц_PROPN', 0.5254683494567871),
 ('брелин_PROPN', 0.5171799659729004),
 ('97-летие_NOUN', 0.5084720849990845),
 ('митинод_NOUN', 0.505733072757721),
 ('демонстрация_NOUN', 0.5037501454353333),
 ('протест_NOUN', 0.5021286606788635),
 ('санкционировать_ADJ', 0.49920910596847534)]

In [17]:
vocabulary = w2v.vocab
len(vocabulary)

249318

In [18]:
len(set([item.split('_')[0] for item in vocabulary.keys()]))

215875

In [21]:
w2v.get_vector('любовь_NOUN').shape

(300,)

In [30]:
# уникальные слова без тэгов
words = list(set([item.split('_')[0] for item in vocabulary.keys()]))

In [32]:
len(words)

215875

In [33]:
import numpy as np 
from tqdm import tqdm_notebook

X = np.zeros([len(words) + 2, 300])

uniq_words = ["#PAD#", "#UNK#"]

i = 2
for word in tqdm_notebook(vocabulary.keys()):
    cur_word = word.split('_')[0]
    if cur_word in uniq_words:
        continue 
       
    uniq_words.append(cur_word)
    X[i] = w2v.get_vector(word)
    i += 1
    if i % 10000 == 0:
        print(i)

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000


In [34]:
X.shape

(215877, 300)

In [36]:
np.save('emb.npy', X)

In [39]:
len(uniq_words) == X.shape[0]

True

In [40]:
dct = {uniq_words[i] : i for i in range(len(uniq_words))}

In [44]:
import pickle

with open('dct.pickle', 'wb') as f:
    pickle.dump(dct, f)

In [47]:
!ls -lh

total 2854440
drwx------@ 6 fulyankin  staff   204B 14 сен 13:42 184
-rw-r--r--  1 fulyankin  staff   7,6K 14 сен 15:51 Untitled.ipynb
-rw-r--r--  1 fulyankin  staff   7,0M 14 сен 15:52 dct.pickle
-rw-r--r--  1 fulyankin  staff   494M 14 сен 15:49 emb.npy
-rw-r--r--@ 1 fulyankin  staff    42K 14 сен 15:11 my_base_file.ipynb
-rw-r--r--@ 1 fulyankin  staff   893M 14 сен 13:47 ria_news_2018.pickle


## Данные

In [94]:
import pickle

# Подгружаем все данные, что у нас есть 
with open('ria_news_2018.pickle', 'rb') as f:
    ria_news = pickle.load(f)

In [101]:
ria_news[0]

{'category': 'В мире',
 'date': '2018-12-31',
 'day': '31',
 'href': '/20181231/1548961410.html',
 'image': 'https://cdn25.img.ria.ru/images/153303/06/1533030626_0:247:3501:2216_1036x0_80_0_0_291c92f1d31f7b2a613588e6a96164cb.jpg',
 'keywords': 'биньямин нетаньяху, израиль, в мире',
 'month': '12',
 'shows': '728',
 'snippet': 'Премьер-министр Израиля Биньямин Нетаньяху не намерен уходить в отставку в случае предъявления ему обвинений в коррупции, сообщает израильский телеканал i24... РИА Новости, 31.12.2018',
 'tags': 'Биньямин Нетаньяху, Израиль, В мире',
 'text': 'МОСКВА, 31 дек - РИА Новости. Премьер-министр Израиля Биньямин Нетаньяху не намерен уходить в отставку в случае предъявления ему обвинений в коррупции, сообщает израильский телеканал i24 News. В начале декабря полиция рекомендовала генпрокуратуре предъявить обвинения в коррупции Нетаньяху и его супруге Саре по итогам расследования дела об их отношениях с руководством крупнейшей в стране телекоммуникационной компании "Безек"

In [ ]:
set([item['category'] for item in ria_news])

In [ ]:
len([item for item in ria_news if item['category'] == ''])

In [100]:
len([item for item in ria_news if item['category'] != ''])

32896

In [ ]:
ria_news = [item for item in ria_news if item['category'] != '']